In [1]:
from __future__ import print_function

import sys
sys.path.append('../')

from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import cv2

import h5py
import os
import time

basedir = '/mnt/dataB/cxr8'
dir_output = '/mnt/dataB/cxr8/output'

import keras
from keras import layers
from keras import models

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


Using TensorFlow backend.


In [2]:
start_time = time.time()

# Open hdf5 file:
with h5py.File(basedir + '/data/cxr8_512_1ch_mass.hdf5', 'r') as hdf:
    X = hdf['X'][:]
    y = hdf['y'][:]
    
'''with h5py.File(basedir + '/data/cxr8_224_3ch_pno.hdf5', 'r') as hdf:
    X = hdf['x_train'][:]
    y = hdf['y_train'][:]'''

print('--- %s seconds ---' % (time.time() - start_time))

--- 53.36891770362854 seconds ---


In [4]:
# For 512 x 512 images (or 1 channel):
prop_test = 0.10
lim_val = int(X.shape[0]*prop_test)*2
lim_test = int(X.shape[0]*prop_test)

x_train = X[:-lim_val,:,:]
x_val = X[-lim_val:-lim_test,:,:]
x_test = X[-lim_test:,:,:]

y_train = y[:-lim_val]
y_val = y[-lim_val:-lim_test]
y_test = y[-lim_test:]

x_train = np.expand_dims(x_train, axis=3)
x_val = np.expand_dims(x_val, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [16]:
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
img_channels = x_train.shape[3]
input_shape = (img_rows, img_cols, img_channels)

#nb_classes = y_train.shape[1]
nb_classes = 1

In [18]:
from addon.jeremy_contrib.nasnet import NASNet

model = NASNet(input_shape=(512,512,1),
       penultimate_filters=4032,
       nb_blocks=6,
       stem_filters=96,
       skip_reduction=False,
       use_auxiliary_branch=False,
       filters_multiplier=2,
       dropout=0.0,
       weight_decay=5e-4,
       include_top=True,
       weights=None,
       input_tensor=None,
       pooling=None,
       classes=1,
       default_size=512)

ModuleNotFoundError: No module named 'addon'

In [19]:
from addon.jeremy_contrib.densenet import DenseNet

model = DenseNet(nb_dense_block=4,
         growth_rate=32,
         nb_filter=64,
         reduction=0.0,
         dropout_rate=0.0,
         weight_decay=1e-4,
         classes=1,
         weights_path=None)

ModuleNotFoundError: No module named 'addon'

In [ ]:
from addon.jeremy_contrib.nasnet import NASNetMobile

# env 210 sec/epoch with gpu=4 and batch_size=64
# env X sec/epoch with gpu=4 and batch_size=32
# env X sec/epoch with gpu=4 and batch_size=16

# 4'233'459 trainable params

model = NASNetMobile(input_shape=(512, 512, 1),
                 dropout=0.5,
                 weight_decay=4e-5,
                 use_auxiliary_branch=False,
                 include_top=True,
                 weights=None,
                 input_tensor=None,
                 pooling=None,
                 classes=1)

In [5]:
from addon.jeremy_contrib.xception import Xception

# env 70 sec/epoch with gpu=4 and batch_size=64
# env 85 sec/epoch with gpu=4 and batch_size=32
# env 95 sec/epoch with gpu=4 and batch_size=16

# 20'808'425 trainable params

model = Xception(include_top=True,
                 weights=None,
                 input_tensor=None,
                 input_shape=input_shape,
                 pooling=None,
                 classes=nb_classes)

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 1)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 255, 255, 32)  288         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 255, 255, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 255, 255, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

### Training the model

In [7]:
# If we want to load previous (best) weights to start training our model:
#model.load_weights(weights_file_orig)

################################################################################
from keras.utils.training_utils import multi_gpu_model
model = multi_gpu_model(model, gpus=4)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

print('Model compiled!')

Model compiled!


In [8]:
# Parameters to set:
###
batch_size = 16
epochs = 300

#weights_file_orig = 'xception_cxr8_512_mass_v3-0.h5'
weights_file_new = 'xception_cxr8_512_mass_v4-1_xception.h5'

In [ ]:
################################################################################
from keras.preprocessing import image
generator = image.ImageDataGenerator(rotation_range=30,
                                     #width_shift_range=5. / 32,
                                     #height_shift_range=5. / 32,
                                     zoom_range=[0.85, 1.15],
                                     horizontal_flip=True)

generator.fit(x_train, seed=0)

print('Data augmentation fitted!')

###
from keras import callbacks
lr_reducer = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         factor=np.sqrt(0.1),
                                         cooldown=0,
                                         patience=20,
                                         min_lr=0.5e-6)

early_stopper = callbacks.EarlyStopping(monitor='val_acc',
                                        min_delta=1e-4,
                                        patience=70)

# We save the new weight file, containing the best model on the validation set:
model_checkpoint = callbacks.ModelCheckpoint(weights_file_new,
                                             monitor='val_acc',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='auto')

callbacks = [lr_reducer, early_stopper, model_checkpoint]

################################################################################
print('Start training the model.')
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=(x_val, y_val),
                    verbose=2)

Data augmentation fitted!
Start training the model.
Epoch 1/300


In [8]:
model.load_weights(weights_file_new)

In [12]:
scores = model.evaluate(x_test, y_test,
                        batch_size=batch_size)

print('Testing set, loss : ', scores[0])
print('Testing set, accuracy : ', scores[1])

y_pred = model.predict(x_test)

1149/1149 [==============================] - 8s     
Testing set, loss :  0.70503099666
Testing set, accuracy :  0.524804177753


In [9]:
# Get the initial model architecture from multi_gpu compression:
model = model.get_layer('xception') # Or any other model name
#model.summary()

In [11]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

print('Model compiled!')

Model compiled!


### Evaluate model metrics

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_bin = y_pred>0.5

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred_bin)

# Figure:
fig, ax = plt.subplots(figsize=(5, 5))

sns.set(style="white")

ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

plt.xlabel('predicted label (PNO)')
plt.ylabel('true label (PNO)')
plt.tight_layout()
#plt.savefig(str(dir_output + os.path.sep + 'cxr8_conf_mat_pno_pno_512_v3.png'), dpi=300)

plt.show()

In [ ]:
def Metrics(y_true, y_pred):
    
    TruePositive = sum((y_true == 1) & (y_pred == 1))
    FalsePositive = sum((y_true == 0) & (y_pred == 1))
    TrueNegative = sum((y_true == 0) & (y_pred == 0))
    FalseNegative = sum((y_true == 1) & (y_pred == 0))
    
    Sensitivity = TruePositive/(TruePositive + FalseNegative)
    Specificity = TrueNegative/(TrueNegative + FalsePositive)
    PPV = TruePositive/(TruePositive + FalsePositive)
    NPV = TrueNegative/(TrueNegative + FalseNegative)
    
    return(Sensitivity, Specificity, PPV, NPV)

Sensitivity, Specificity ,PPV, NPV = Metrics(y_test, y_pred_bin[:,0])

print('Sensitivity: %.2f' %(Sensitivity * 100))
print('Specificity: %.2f' %(Specificity * 100))
print('Positive Predicting Value: %.2f' %(PPV * 100))
print('Negative Predicting Value: %.2f' %(NPV * 100))

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Figure:
plt.figure(figsize=(5,5))
sns.set(style="white")
plt.plot(fpr, tpr, color='darkorange', lw=2, linestyle='--', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], linestyle=':', color='gray', label='Random guessing')
plt.plot([0, 0, 1],[0, 1, 1],linestyle=':', color='black', label='Perfect performance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic, on TESTING set')
plt.legend(loc="lower right")
#plt.savefig(str(dir_output + os.path.sep + 'cxr8_ROC_testing_pno_512_v3.png'), dpi=300)
plt.show()

## Visualisation

In [ ]:
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.measure import block_reduce
import cv2

### Visualise native chest xray

In [ ]:
### class_idx = 0
indices = np.where(y_test[:] == 1.)[0]

# pick some random input from here.
patient = 2 #25
idx = indices[patient]
idx=99 #45

'''Good examples:
    45,
   Medium... 126 
   Conter-ex: 127'''

print('Diagnosis: %i' %(y_test[idx]))
print('Predicted: %.3f' %(y_pred[idx][0]))

fig, ax = plt.subplots(1,2, figsize=(20,20))
ax[0].imshow(x_test[idx][..., 0], cmap='gray')
ax[0].axis('off')
ax[1].imshow(x_test[idx][..., 0]*-1, cmap='gray')
ax[1].axis('off')
plt.show()


In [ ]:
true_0 = np.where(y_test==0)[0]
pred_0 = np.where(y_pred<0.5)[0]
true_1 = np.where(y_test==1)[0]
pred_1 = np.where(y_pred>0.5)[0]

indices_TP = np.intersect1d(true_1, pred_1) 
indices_FP = np.intersect1d(true_0, pred_1)
indices_TN = np.intersect1d(true_0, pred_0)
indices_FN = np.intersect1d(true_1, pred_0)

In [ ]:
nb_img_to_show = 20

fig, ax = plt.subplots(nb_img_to_show,2, figsize=(10,(5*nb_img_to_show)))

for i in range(nb_img_to_show):  
    
    idx = indices_TP[i+20]

    #print('Diagnosis: %i' %(y_test[idx]))
    #print('Predicted: %.3f' %(y_pred[idx][0]))
    
    ax[i,0].imshow(x_test[idx][..., 0], cmap='gray')
    ax[i,0].axis('off')
    ax[i,0].set_title('Diagnosis (idx): %i (%i)' %(y_test[idx], idx)) 
    ax[i,1].imshow(x_test[idx][..., 0]*-1, cmap='gray')
    ax[i,1].axis('off')
    ax[i,1].set_title('Predicted: %.3f' %(y_pred[idx][0])) 

plt.show()

### Explore model output (saliency maps, grad-CAM maps)

In [ ]:
from vis.visualization import visualize_saliency
from vis.visualization import visualize_cam
from vis.utils import utils
from keras import activations

layer_idx = utils.find_layer_idx(model, 'predictions')
class_idx = 0

# select the idx of the patient to explore:
#idx = 1064

In [ ]:
#model.layers[layer_idx].activation = activations.softmax
#model.layers[layer_idx].activation = activations.sigmoid
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

# Saliency:  
grads_saliency_vanilla = visualize_saliency(model, layer_idx, filter_indices=class_idx,
                                    seed_input=x_test[idx], backprop_modifier=None)
grads_saliency_guided = visualize_saliency(model, layer_idx, filter_indices=class_idx,
                                    seed_input=x_test[idx], backprop_modifier='guided')
grads_saliency_relu = visualize_saliency(model, layer_idx, filter_indices=class_idx,
                                    seed_input=x_test[idx], backprop_modifier='relu')
# grad-CAM:
grads_cam_vanilla = visualize_cam(model, layer_idx, filter_indices=class_idx,
                          seed_input=x_test[idx], backprop_modifier=None)  
grads_cam_guided = visualize_cam(model, layer_idx, filter_indices=class_idx,
                          seed_input=x_test[idx], backprop_modifier='guided')  
grads_cam_relu = visualize_cam(model, layer_idx, filter_indices=class_idx,
                          seed_input=x_test[idx], backprop_modifier='relu')

In [ ]:
plt.figure(figsize=(20,10))

f, ax = plt.subplots(1, 4)
ax[0].imshow(x_test[idx][..., 0], cmap='gray')
ax[1].set_title('vanilla')    
ax[1].imshow(grads_saliency_vanilla, cmap='jet')
ax[2].set_title('guided')    
ax[2].imshow(grads_saliency_guided, cmap='jet')
ax[3].set_title('relu')    
ax[3].imshow(grads_saliency_relu, cmap='jet')

f, ax = plt.subplots(1, 4)
ax[0].imshow(x_test[idx][..., 0], cmap='gray')
ax[1].set_title('vanilla')    
ax[1].imshow(grads_cam_vanilla, cmap='jet')
ax[2].set_title('guided')    
ax[2].imshow(grads_cam_guided, cmap='jet')
ax[3].set_title('relu')    
ax[3].imshow(grads_cam_relu, cmap='jet')

plt.show()

In [ ]:
block_size = 25

img = rgb2gray(grads_saliency_vanilla)
img2 = block_reduce(img, block_size=(block_size,block_size))
img3 = cv2.resize(img2, (img.shape[1], img.shape[0]))

plt.figure(figsize=(10,10))
plt.title('Diagnosis: %i,     Predicted: %.3f' %(y_test[idx], y_pred[idx][0]))
plt.imshow(x_test[idx][..., 0], cmap='gray')
plt.imshow(img3**3, alpha=0.3, cmap='hot')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,20))
ax[0].imshow(x_test[idx][..., 0], cmap='gray')
ax[0].axis('off')
ax[1].imshow(x_test[idx][..., 0], cmap='gray')
ax[1].imshow(img3**3, alpha=0.3, cmap='hot')
ax[1].axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Diagnosis: %i,     Predicted: %.3f' %(y_test[idx], y_pred[idx][0]))
plt.imshow(x_test[idx][..., 0], cmap='gray')
plt.imshow(grads_cam_vanilla, alpha=0.3, cmap='hot')
plt.show()